<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-VGG16/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

## Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [2]:
! pip install bing_image_downloader

In [3]:
from bing_image_downloader import downloader

In [45]:
# Query
search_queries=[
    'Nicole Kidman',
    'Hugh Jackman',
    'Mel Gibson',
    'Russell Crowe',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name))

for query in search_queries:
  donwloader(query, n_sample=20, dataset_name='train')
  donwloader(query, n_sample=10, dataset_name='val')
  donwloader(query, n_sample=5, dataset_name='test')

[%] Downloading Images to /content/dataset/train/Nicole Kidman


[!!]Indexing page: 1

[%] Indexed 20 Images on Page 1.


[%] Downloading Image #1 from https://celebmafia.com/wp-content/uploads/2016/04/nicole-kidman-academy-of-country-music-awards-2016-in-las-vegas-1.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://celebmafia.com/wp-content/uploads/2016/09/nicole-kidman-lion-premiere-tiff-in-toronto-9-10-2016-1.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://heightline.com/wp-content/uploads/nicole-kidman.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://www.hawtcelebs.com/wp-content/uploads/2012/10/NICOLE-KIDMAN-at-Nicole-Kidman-Gala-Tribute-at-the-50th-Annual-New-York-Film-Festival-3.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://images2.fanpop.com/images/photos/4800000/Nicole-Kidman-nicole-kidman-4896048-1024-768.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://images5.fanpop.com/image/photos/31800000/Nico

KeyboardInterrupt: 

In [5]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [6]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  185
#train_files:  100
#val_files:  50
#test_files:  35


In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [37]:
batch_size = 32
lr = 0.001
epoch = 10

# Define Image data generator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 100 images belonging to 7 classes.
Found 50 images belonging to 7 classes.
Found 35 images belonging to 7 classes.


In [38]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers except the last Conv. layer
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add the customized classifier
block_5_flatten = Flatten()(base_model.output)
block_5_dense = Dense(256, activation='relu')(block_5_flatten)
block_5_dropout = Dropout(0.6)(block_5_dense)

predictions = Dense(len(search_queries), activation='softmax')(block_5_dropout)

# Build a new model
new_model = Model(inputs=base_model.input, outputs=predictions)
new_model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

## Train and Evaluate the New_Model

In [ ]:
# Train the new model
steps_per_epoch = round(train_generator.samples / batch_size)
validation_steps = round(val_generator.samples / batch_size)

history = new_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch,
    validation_data=val_generator,
    validation_steps=validation_steps
)

In [43]:
# Evaluate the model
test_loss, test_acc = new_model.evaluate(test_generator)
print('Test accuracy:', test_acc)

2/2 [==============================] - 3s 1s/step - loss: 1.9796 - accuracy: 0.1429
Test accuracy: 0.1428571492433548
